## Question 2: Texture Synthesis

first , we import needed libraries and files :

In [ ]:
import numpy as np
import cv2
import q2_funcs


Functions used in the question are implemented in
q2_funcs.py. Implementation of these functions are as follow:


## q2_funcs.py

The first function in the file is `get_random_patch` which gets texture and patch_size
as input and outputs a random patch from texture with the given size:



In [ ]:
import numpy as np
import cv2
import random


def get_random_patch(texture, patch_size):
    x_max, y_max = texture.shape[0] - patch_size[0] - 1, texture.shape[1] - patch_size[1] - 1
    x, y = random.randint(0, x_max), random.randint(0, y_max)
    patch = texture[x:x + patch_size[0], y:y + patch_size[1], :].copy()
    return patch


Second function is `find_patch_from_template` , which is used to find all the
patches from the texture that match a given template. This method takes only templates
of rectangular shape, and is only used when synthesising the first row and
first column of the result :


In [ ]:
def find_patch_from_template(texture, template):
    search_result = cv2.matchTemplate(texture, template, cv2.TM_CCORR_NORMED)
    threshold = 0.95
    while threshold > 0.5:
        match_points_x, match_points_y = np.where(search_result > threshold)
        if match_points_x.size > 0:
            i = random.randint(0, match_points_x.size - 1)
            x, y = match_points_x[i], match_points_y[i]
            return x, y
        threshold -= 0.05


Function begins with a base threshold 0.95, and then iterates
on this value by decreasing it 0.05 and finding all results
which are above the threshold in each iteration. At any time that
there is at least one result, iteration would be stopped and location
of one of the results will be randomly returned.


From here, we have functions related to finding min-cut. Function
`find_path` takes a matrix of weights as input and outputs a matrix
with elements showing the min-cut, derived from these weights:


In [ ]:
def find_path(weights):
    result = np.zeros((weights.shape[0], weights.shape[1], 3))
    index = np.where(weights[-1, :] == np.min(weights[-1, :]))[0][0]
    result[-1, 0:index, :] = 1
    for i in reversed(range(weights.shape[0] - 1)):
        if index == 0:
            if weights[i, 0] > weights[i, 1]:
                index = 1
                result[i, 0, :] = 1
        elif index == weights.shape[1] - 1:
            if weights[i, index] > weights[i, index - 1]:
                index = index - 1
            result[i, 0:index, :] = 1
        else:
            t = min(weights[i, index - 1], weights[i, index], weights[i, index + 1])
            if weights[i, index - 1] == t:
                index = index - 1
            elif weights[i, index + 1] == t:
                index = index + 1
            result[i, 0:index, :] = 1
    return result


In result matrix, all the cells on the left side of min-cut have
value 1 and cells on the right side of min-cut have value 0. Setting
 result in this way makes further computations simpler.

Next function is `find_min_cut`, which takes two templates as
input and returns the min-cut of their difference:

In [ ]:
def find_min_cut(template1, template2):
    difference = np.sum(np.square(template1 - template2), axis=2, keepdims=True).reshape(
        (template1.shape[0], template1.shape[1]))
    weights = np.zeros(difference.shape)
    weights[0, :] = difference[0, :].copy()
    for j in range(1, difference.shape[0]):
        for i in range(difference.shape[1]):
            if i == 0:
                weights[j, i] = difference[j, i] + min(weights[j - 1, i], weights[j - 1, i + 1])
            elif i == difference.shape[1] - 1:
                weights[j, i] = difference[j, i] + min(weights[j - 1, i], weights[j - 1, i - 1])
            else:
                weights[j, i] = difference[j, i] + min(weights[j - 1, i], weights[j - 1, i + 1], weights[j - 1, i - 1])
    result = find_path(weights)
    return result


After calculating the SSD of two templates, a weights matrix is
calculated and given as input to function `find_path`, described
before.


Now after implementing functions needed for filling the first
 row, we implement function `first_row_synthesis`, which does this:

In [ ]:
def first_row_synthesis(img, texture, patch_size, overlap):
    first_patch = get_random_patch(texture, patch_size)
    img[0:patch_size[0], 0:patch_size[1], :] = first_patch
    number_of_patches = int((img.shape[1] - patch_size[1]) / (patch_size[1] - overlap))
    cut_from_end = patch_size[1]
    for i in range(number_of_patches):
        template_x = (i + 1) * (patch_size[0] - overlap)
        template = img[0:patch_size[1], template_x:template_x + overlap, :].copy()
        x, y = find_patch_from_template(texture[:, 0:-cut_from_end, :], template)
        template2 = texture[x:x + patch_size[0], y:y + overlap, :].copy()
        min_cut = find_min_cut(template, template2)
        final_template = (min_cut * template + (1 - min_cut) * template2).copy()
        img[0:patch_size[0], template_x:template_x + overlap, :] = final_template.copy()
        patch = texture[x:x + patch_size[0], y + overlap:y + patch_size[1], :].copy()
        img[0:patch_size[0], template_x + overlap:template_x + patch_size[0], :] = patch.copy()

Function starts by putting a random patch on the top left of the
empty image, and fills the rest of the row by matching a 100 in 200
template from the end of the row with texture, selecting one of the
matches randomly, calculating the min-cut of the matched template
with the template taken from the row, and attaching a patch to the
end of the row using min-cut and matched template.

## Synthesising other rows

Functions implemented from here are used for synthesising other rows.


The `find_patch_from_L_template` takes a texture, template and a
mask ( with L form ) and uses template matching to find all
templates matching with the given one. Implementation is similar to
`find_patch_from_template` function described before :


In [ ]:
def find_patch_from_L_template(texture, template, mask):
    search_result = cv2.matchTemplate(texture, template, cv2.TM_CCORR_NORMED, mask=mask)
    threshold = 0.95
    while threshold > 0.5:
        match_points_x, match_points_y = np.where(search_result > threshold)
        if match_points_x.size > 0:
            i = random.randint(0, match_points_x.size - 1)
            x, y = match_points_x[i], match_points_y[i]
            return x, y
        threshold -= 0.05

function `find_L_min_cut` takes as input two templates, template1
 and template2 of the form 'L', and a value n=overlap, indicating the
  thickness of 'L', and returns the L-min-cut of templates.
  For implementing this, we first find the min-cut of the first n
  rows of each template and store it in a matrix `top_min_cut`, and
  similarly do this for the first n columns of templates and store it
   in a matrix `left_min_cut`, and finally compute the logical or of
   these two matrices with the first n rows and n columns of the result
    matrix , which is initially all zero. This will result a matrix
    having 1 in cells above the min-cut, and 0 below the min-cut:

In [ ]:
def find_L_min_cut(template1, template2, overlap):
    result = np.zeros(template1.shape)
    top_min_cut = find_min_cut(template1[0:overlap, :, :].transpose(1, 0, 2),
                               template2[0:overlap, :, :].transpose(1, 0, 2)).transpose(1, 0, 2)
    left_min_cut = find_min_cut(template1[:, 0:overlap, :], template2[:, 0:overlap, :])
    result[0:overlap, :, :] = np.logical_or(result[0:overlap, :, :], top_min_cut)
    result[:, 0:overlap, :] = np.logical_or(result[:, 0:overlap, :], left_min_cut)
    return result

Now that we have implemented matching and min-cut functions for next
rows, we can put all these together and implement the function `synthesis_row_i`:

In [ ]:
def synthesis_row_i(img, texture, patch_size, overlap, i):
    x = i * (patch_size[0] - overlap)
    template = img[x:x + overlap, 0:patch_size[1], :].copy()
    x1, y1 = find_patch_from_template(texture[0:-patch_size[0], :, :], template)
    template2 = texture[x1:x1 + overlap, y1:y1 + patch_size[1], :].copy()
    min_cut = find_min_cut(template.transpose(1, 0, 2), template2.transpose(1, 0, 2)).transpose(1, 0, 2)
    final_template = (min_cut * template + (1 - min_cut) * template2).copy()
    img[x:x + overlap, 0:patch_size[1], :] = final_template.copy()
    patch = texture[x1 + overlap:x1 + patch_size[0], y1:y1 + patch_size[1], :].copy()
    print(patch.shape)
    img[x + overlap:x + patch_size[0], 0:patch_size[1], :] = patch.copy()
    ''' synthesising the rest of the row '''
    mask = np.zeros((patch_size[0], patch_size[1], 3), dtype='uint8')
    mask[0:overlap, :, :] = 1
    mask[:, 0:overlap, :] = 1
    number_of_patches = int((img.shape[1] - patch_size[1]) / (patch_size[1] - overlap))
    for j in range(number_of_patches):
        template_x = (j + 1) * (patch_size[0] - overlap)
        template = img[x:x + patch_size[1], template_x:template_x + patch_size[0], :].copy()
        x1, y1 = find_patch_from_L_template(texture[0:-patch_size[0], 0:-patch_size[0], :], template, mask)
        template2 = texture[x1:x1 + patch_size[1], y1:y1 + patch_size[0], :].copy()
        min_cut = find_L_min_cut(template, template2, overlap)
        patch = (min_cut * template + (1 - min_cut) * template2).copy()
        img[x:x + patch_size[0], template_x:template_x + patch_size[1], :] = patch.copy()

implementation of this function is just similar to `first_row_synthesis`,
only difference is that it uses the two above functions for matching
templates and finding min-cut.

Finally, we have implemented everything required for synthesising next rows.
Function `next_rows_synthesis` takes an image, a texture, patch size and overlap
as input and synthesises other rows on the image with texture, assuming
that the first row is already synthesised.

In [ ]:
def next_rows_synthesis(img, texture, patch_size, overlap):
    num_of_iterations = int(img.shape[0] / (patch_size[0] - overlap) - 1)
    for i in range(1, num_of_iterations):
        synthesis_row_i(img, texture, patch_size, overlap, i)


The final function `apply_texture_synthesis` takes a texture, the result
image shape, patch size and overlap value as input and outputs an image
containing both synthesised and original texture:

In [ ]:
def apply_texture_synthesis(texture, result_shape, patch_size, overlap):
    result = np.zeros(result_shape, dtype='uint8')
    first_row_synthesis(result, texture, patch_size, overlap)
    next_rows_synthesis(result, texture, patch_size, overlap)
    final_result = np.zeros((result_shape[0], result_shape[1] + texture.shape[1] + overlap, result_shape[2]),
                            dtype='uint8')
    final_result[:, :result_shape[1], :] = result
    final_result[:texture.shape[0], result_shape[1] + overlap:, :] = texture
    return final_result

## Using Previous function for synthesising given textures
In main file, we just open texture images, iterate over them and perform
 texture synthesis implemented in previous method :

In [ ]:
import cv2
import q2_funcs

input_paths = ["inputs/texture03.jpg", "inputs/texture08.jpg",
               "inputs/texture6.jpg", "inputs/texture10.jpg"]
output_paths = ["outputs/res11.jpg", "outputs/res12.jpg",
                "outputs/res13.jpg", "outputs/res14.jpg"]

result_shape = (2500, 2500, 3)
patch_size = (200, 200)
overlap = 100

for i in range(len(input_paths)):
    print(i)
    print(input_paths[i])
    texture = cv2.imread(input_paths[i])
    print(texture.shape)
    result = q2_funcs.apply_texture_synthesis(texture, result_shape, patch_size, overlap)
    cv2.imwrite(output_paths[i], result)